In [20]:
import boto3

import json
import numpy as np
import pandas as pd
import time
import uuid

personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client(service_name='personalize-events')


In [21]:
session_dict = {}

In [22]:
# First get 

# Tracking ID
# d8f48295-c014-4f63-a382-11d7fc98375c
# Tracker name
# interpret-event-tracker
# Tracker ARN
# arn:aws:personalize:us-east-1:111652037296:event-tracker/2c09af72


def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'EVENT_TYPE',
        'properties': event_json
        }]
    )

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the title of the movie for the header of the column
    movie_title_clicked = get_movie_title(movie_to_click)
    # Interact with the movie
    send_movie_click(USER_ID=str(user_id), ITEM_ID=movie_to_click)
    # Sleep for 2 seconds
    time.sleep(2)
    # Get new recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        title = get_movie_title(item['itemId'])
        recommendation_list.append(title)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_title_clicked])
    # Add this dataframe to the old one
    recommendations_df = recommendations_df.join(new_rec_DF)
    return recommendations_df

In [64]:
generated_data_df = pd.read_csv('generated_data.csv')

c = 0 

for i, j in generated_data_df.iterrows(): 
    
    c = c + 1
    
    if(c>30):
        break
        
    print('->'+str(j['USER_ID'])+'<-')



->17872<-
->17872<-
->17872<-
->17872<-
->17872<-
->18627<-
->18627<-
->18627<-
->18627<-
->10190<-
->10190<-
->10190<-
->29070<-
->29070<-
->29070<-
->29070<-
->25608<-
->25608<-
->25608<-
->25608<-
->25608<-
->22037<-
->22037<-
->22037<-
->10800<-
->10800<-
->10800<-
->10800<-
->10800<-
->24619<-


In [83]:
def load_recipe(filename):
    """Load a single recipe collection from disk."""
    with open(filename, 'r') as f:
        recipes = json.load(f)
    print('Loaded {:,} recipes from {}'.format(len(recipes), filename))
    return recipes

recipe = load_recipe('recipes/recipes_raw_nosource_ar.json')

i = 0
for key, value in recipe.items():
    print(json.dumps(key, indent=4))
    print(value['title'])
    i += 1
    if (i>10):
        break

Loaded 39,802 recipes from recipes/recipes_raw_nosource_ar.json
"rmK12Uau.ntP510KeImX506H6Mr6jTu"
Slow Cooker Chicken and Dumplings
"5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm"
Awesome Slow Cooker Pot Roast
"clyYQv.CplpwJtjNaFGhx0VilNYqRxu"
Brown Sugar Meatloaf
"BmqFAmCrDHiKNwX.IQzb0U/v0mLlxFu"
Best Chocolate Chip Cookies
"N.jCksRjB4MFwbgPFQU8Kg.yF.XCtOi"
Homemade Mac and Cheese Casserole
"kq.naD.8G19M4UU9dVvJgHtpfo.l/eC"
Banana Banana Bread
"lYrgWNn00EXblOupzM3tL0jGr9O0CB2"
Chef John's Fisherman's Pie
"Fu0DgGYFUGwc0BBlN6r20o/ihOVs5bO"
Mom's Zucchini Bread
"MBRNtqELRRuv8zJH4k7Aba2bmIc2A3C"
The Best Rolled Sugar Cookies
"ZPyPoMiNvgAfrKcRpH9FEYV/XsPZBsW"
Singapore Chili Crabs
"ATMxl11LFhuvTbTi9.kX4nhYE50HeJ."
Downeast Maine Pumpkin Bread


In [161]:
import sys

generated_data_df = pd.read_csv('generated_data.csv')

c = 0 
campaign_arn = 'arn:aws:personalize:us-east-1:111652037296:campaign/interpret-campaign-aws-hrnn-08549'
tracking_id = 'f71c2423-50f0-4b64-9e44-2b43d3f9aef7'
start = int(time.time())

for i, j in generated_data_df.iterrows(): 
    
    print('_'*31)
    
    c = c + 1
    
    if(c>1):
        break
        
    get_recommendations_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId=str(j['USER_ID']))
    
    item_list = get_recommendations_response['itemList']
    
    item_list_list = []
    
    for item in item_list:
        item_list_list.append(item['itemId'])
    
    #get_personalized_ranking_response = personalize_runtime.get_personalized_ranking(campaignArn=campaign_arn, inputList=item_list_list, userId=str(j['USER_ID']))
    #print(get_personalized_ranking_response)
    
    states=['dislike', 'open', 'sharedf', 'like', 'sharedt', 'sharede', 'addc', 'addposcom', 'activityc', 'purchased', 'done']
    
    user_actions = {
    "dislike":"0",
    "open":"10",
    "sharedf":"20",
    "like":"30",
    "sharedt":"40",
    "sharede":"50",
    "addc":"60",
    "addposcom":"70",
    "activityc":"80",
    "purchased":"90",
    "done":"100"
    }
   
    for item in item_list_list:
        for key, value in recipe.items():
            if(key == item):
                #print(item)
                print('\"'+value['title']+'\",')
                
        for u in user_actions:
            personalize_events.put_events(
                trackingId = tracking_id ,
                userId= str(j['USER_ID']),
                sessionId = str(uuid.uuid1()),
                eventList = [{
                    'eventId': 'event1' + str(uuid.uuid1()),
                    'sentAt': start+i*1000,
                    'eventType': u,
                    'properties': json.dumps({
                        'itemId': item,
                        'eventValue': int(user_actions[u])
                        })
                    }, {
                    'eventId': 'event1' + str(uuid.uuid1()),
                    'sentAt': start+i*1000,
                    'eventType': u,
                    'properties': json.dumps({
                        'itemId': item,
                        'eventValue': int(user_actions[u])
                        })
                    }]
            )
        

_______________________________
"Moist Persimmon Cookie",
"Mexican Rice Salad",
"Glazed Corned Beef",
"Ham Ball",
"Easy Southern Biscuits",
"Scalloped Oysters",
"Fettuccini with Mushroom, Ham and Rose Sauce",
"Easy Tuna Casserole",


KeyboardInterrupt: 

In [179]:
from tqdm import tqdm

generated_users_df = pd.read_csv('generated_users.csv')
c = 0 
campaign_arn = 'arn:aws:personalize:us-east-1:111652037296:campaign/interpret-campaign-aws-hrnn-08549'
tracking_id = 'f71c2423-50f0-4b64-9e44-2b43d3f9aef7'
start = int(time.time())

states=['dislike', 'open', 'sharedf', 'like', 'sharedt', 'sharede', 'addc', 'addposcom', 'activityc', 'purchased', 'done']

user_actions = {
    "dislike":"0",
    "open":"10",
    "sharedf":"20",
    "like":"30",
    "sharedt":"40",
    "sharede":"50",
    "addc":"60",
    "addposcom":"70",
    "activityc":"80",
    "purchased":"90",
    "done":"100"
}


make_popular_list = ["Mexican Rice Salad",
"Glazed Corned Beef",
"Scalloped Oysters",
"Fettuccini with Mushroom, Ham and Rose Sauce",
"Easy Peanut Blossoms",
"Ham Ball",
"Genoese Sponge",
"Garlic and Leek Ditalini",
"German French Toast",
"Easy Tuna Casserole",
"Moist Persimmon Cookie",
"My Country Style Steak",
"Punch Bowl Cookies",
"Honey Mustard Drizzle",
"Poke-Style Wings",
"Foglie Chef Doogie's D'Autunno with Chicken and Portobella Mushrooms",
"Salmon Supreme",
"Keerti's Karwari Prawns",
"Shrimp Alfredo Pasta",
"Citrus Honey Brined Smoked Turkey",
"Lucy's Tomato and Peach Chutney",
"Baked French Fries II",
"Johnny Marzetti Casserole",
"Sheila's Perfect Sangria",
"Egg Salad I"]

for i in tqdm(range(0, 10000)):
    for item_in_make_popular_list in make_popular_list:
        for key, value in recipe.items():
            if 'title' in value.keys():
                title = value['title']
                if (title == item_in_make_popular_list):
                    #print(item_in_make_popular_list)
                    #print(str(key))

                    #Idea is to take a random user and like these items over and over again 

                    session_id = str(uuid.uuid1())
                    user_id = str(generated_users_df.sample(n=1)['USER_ID'].values[0])
                    user_action_selected_1 = 'done'
                    user_action_selected_2 = 'purchased'                
                    #item_in_make_popular_list

                    personalize_events.put_events(
                        trackingId = tracking_id ,
                        userId = user_id,
                        sessionId = session_id,
                        eventList = [{
                            'eventId': 'event1' + session_id,
                            'sentAt': start+i*1000,
                            'eventType': user_action_selected_1,
                            'properties': json.dumps({
                                'itemId': item,
                                'eventValue': int(user_actions[user_action_selected_1])
                                })
                            }, {
                            'eventId': 'event2' + session_id,
                            'sentAt': start+i*1000,
                            'eventType': user_action_selected_2,
                            'properties': json.dumps({
                                'itemId': item,
                                'eventValue': int(user_actions[user_action_selected_2])
                                })
                            }]
                        )            
                    break

    

  8%|▊         | 819/10000 [11:14<1:54:01,  1.34it/s]

KeyboardInterrupt: 